In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import scanpy as sc

import SEACells as sea 

import datatable as dt

from sklearn.preprocessing import OneHotEncoder

/home/oem/anaconda3/envs/pyro/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
adata = sc.read_h5ad('GSE194122_openproblems_neurips2021_multiome_BMMC_processed.h5ad')

In [ ]:
A=dt.fread('multiome_neurips21_B1_soft_assignments.txt.gz', header=True).to_pandas().set_index('index')
A.columns = ['multiome_neurips21_B1_{}'.format(i) for i in A.columns]

metacells = A.idxmax(axis=1)

In [ ]:

adata = adata[metacells.index.tolist()]
adata.obs['metacells'] = metacells[adata.obs_names]


adata_metacells = sea.core.summarize_by_SEACell(adata, SEACells_label='metacells', celltype_label='cell_type', summarize_layer='counts')

adata_metacells.obs['cell_type'] = adata.obs.groupby('metacells').apply(
    lambda x: x['cell_type'].mode().iloc[0]
    ).loc[adata_metacells.obs_names]

adata_metacells.obs['Batch'] = 'B1'

In [ ]:
adata_metacells.X = adata_metacells.layers['raw'].copy()

sc.pp.normalize_total(adata_metacells, target_sum=1e4)
sc.pp.log1p(adata_metacells)
sc.pp.highly_variable_genes(adata_metacells, n_top_genes=2000, inplace=True)

sc.tl.pca(adata_metacells, use_highly_variable=True)
sc.pp.neighbors(adata_metacells, use_rep='X_pca')
sc.tl.umap(adata_metacells)

sc.pl.umap(adata_metacells, color='cell_type', palette='Set1', s=20)


In [ ]:
adata_metacells.X = adata_metacells.layers['raw'].copy()
adata_metacells.write_h5ad('multiome_neurips21_B1_metacells.h5ad')